# Experimento Mixto Avanzado: Sequential Transfer Learning (Multi-Model)

Este notebook implementa una estrategia de **Sequential Transfer Learning** para detectar corrosión.
El proceso consta de dos fases para cada modelo (Nano, Medium, Large):
1.  **Fase 1 (Pre-entrenamiento Aéreo):** El modelo aprende características generales de corrosión usando un dataset aéreo.
2.  **Fase 2 (Fine-Tuning Submarino):** El modelo adapta su conocimiento al dominio submarino (objetivo), con una tasa de aprendizaje baja y backbone congelado inicialmente.

In [1]:
import shutil
from pathlib import Path
import torch
from ultralytics import YOLO
import os

In [2]:
# --- CONFIGURACIÓN GLOBAL ---
# Asegúrate de que estas rutas existen
ACUATICO_DIR = Path("./dataset_yolo")       
AEREO_DIR = Path("./dataset_aereo")         
OUTPUT_DIR = Path("./modelos_entrenados")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Parámetros Globales
PROJECT_NAME = "Sequential_Transfer_Benchmark"
EPOCHS_AEREO = 30         
EPOCHS_FINE_TUNE = 50     

print(f"\n{'='*60}")
print(f" ESTRATEGIA: SEQUENTIAL TRANSFER LEARNING (MULTI-MODEL)")
print(f" Fase 1: Aprender Corrosión Aérea -> Fase 2: Adaptar a Submarino")
print(f"{'='*60}")

# --- PREPARACIÓN DE YAMLs ---

# 1. Crear YAML Aéreo temporal (CON EL FIX DE VALIDACIÓN)
yaml_aereo = Path("temp_aereo.yaml")
with open(yaml_aereo, "w") as f:
    # TRUCO: Usamos 'train' también como 'val' porque el dataset aéreo no tiene split
    # Esto evita el error "missing path .../val"
    train_path = (AEREO_DIR / "images/train").absolute()
    
    f.write(f"path: {AEREO_DIR.absolute()}\n")
    f.write(f"train: {train_path}\n")
    f.write(f"val: {train_path}\n")  # <--- AQUÍ ESTÁ EL FIX
    f.write("names:\n  0: corrosion")

# 2. Crear YAML Submarino (Usamos tu dataset limpio original)
yaml_acuatico = Path("temp_acuatico.yaml")
with open(yaml_acuatico, "w") as f:
    # Asumimos que dataset_yolo SÍ tiene estructura correcta (images/train, images/val)
    # Si usaste K-Fold antes, dataset_yolo original puede no tener 'val' fijo.
    # Usaremos el dataset_yolo_aug/dataset.yml si existe, o apuntamos al original.
    
    # Opción A: Usar el dataset original directo (YOLO hará un split automático si solo hay train)
    f.write(f"path: {ACUATICO_DIR.absolute()}\n")
    f.write("train: images/train\n")
    
    # Verificamos si existe val, si no, usamos train (YOLO dividirá auto)
    if (ACUATICO_DIR / "images/val").exists():
        f.write("val: images/val\n")
    elif (ACUATICO_DIR / "images/valid").exists():
        f.write("val: images/valid\n")
    else:
        # Si no hay val, usamos train y dejamos que YOLO se queje o use train
        f.write("val: images/train\n") 
        
    f.write("names:\n  0: corrosion")

print("✅ Archivos YAML de configuración creados.")


 ESTRATEGIA: SEQUENTIAL TRANSFER LEARNING (MULTI-MODEL)
 Fase 1: Aprender Corrosión Aérea -> Fase 2: Adaptar a Submarino
✅ Archivos YAML de configuración creados.


## 1. Entrenamiento Modelo NANO (yolo12n)
**Batch Size:** 32 (Modelo ligero, permite batch alto)

In [3]:
MODEL_ARCH = "yolo12n.pt"
BATCH_SIZE = 32
model_name_clean = "yolo12n"

print(f"\n\n{'#'*60}")
print(f" INICIANDO PROCESO PARA MODELO: {model_name_clean}")
print(f"{'#'*60}")

# --- FASE 1: PRE-ENTRENAMIENTO AÉREO ---
print(f"\n>>> [{model_name_clean}] FASE 1: ENTRENAMIENTO EN DOMINIO AÉREO (FUENTE)")

model_phase1 = YOLO(MODEL_ARCH)
model_phase1.train(
    data=str(yaml_aereo),
    epochs=EPOCHS_AEREO,
    imgsz=640,
    batch=BATCH_SIZE,
    project=PROJECT_NAME,
    name=f"phase1_aerial_{model_name_clean}",
    exist_ok=True,
    verbose=True, 
    optimizer='SGD',
    lr0=0.01
)

# Recuperar los pesos resultantes de la Fase 1
weights_phase1 = Path(PROJECT_NAME) / f"phase1_aerial_{model_name_clean}" / "weights" / "best.pt"
print(f"✅ [{model_name_clean}] Fase 1 Completada. Pesos 'Expertos en Aire' guardados en: {weights_phase1}")

# --- FASE 2: FINE-TUNING SUBMARINO ---
print(f"\n>>> [{model_name_clean}] FASE 2: FINE-TUNING EN DOMINIO SUBMARINO (OBJETIVO)")

if weights_phase1.exists():
    model_phase2 = YOLO(weights_phase1)

    # 3. Entrenar (Fine-Tuning)
    model_phase2.train(
        data=str(yaml_acuatico),
        epochs=EPOCHS_FINE_TUNE,
        imgsz=640,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=f"phase2_underwater_{model_name_clean}",
        exist_ok=True,
        optimizer='SGD',
        lr0=0.005,      # LR Bajo
        lrf=0.1,
        freeze=10,      # Congelar backbone
        augment=True    
    )

    # Guardar resultado final
    final_weights = Path(PROJECT_NAME) / f"phase2_underwater_{model_name_clean}" / "weights" / "best.pt"
    dest = OUTPUT_DIR / f"modelo-mixto-n.pt"
    
    if final_weights.exists():
        shutil.copy(final_weights, dest)
        print(f"\n✅ MODELO MIXTO SECUENCIAL GUARDADO: {dest}")

        # --- VALIDACIÓN FINAL ---
        print(f"\nValidando el nuevo modelo {model_name_clean}...")
        test_yaml = './dataset_yolo/dataset.yml' 
        try:
            metrics = model_phase2.val(split='test', data=test_yaml)
            print(f"[{model_name_clean}] mAP@50 Final: {metrics.box.map50:.4f}")
        except Exception as e:
            print(f"No se pudo validar automáticamente. Error: {e}")
    else:
        print("Error: No se generaron pesos en la Fase 2.")
else:
    print(f"❌ Error Crítico: No se encontró el modelo de la Fase 1 en {weights_phase1}")



############################################################
 INICIANDO PROCESO PARA MODELO: yolo12n
############################################################

>>> [yolo12n] FASE 1: ENTRENAMIENTO EN DOMINIO AÉREO (FUENTE)
New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.228 🚀 Python-3.9.13 torch-2.8.0+cu128 CPU (AMD Ryzen 7 5800X 8-Core Processor)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=temp_aereo.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, i

KeyboardInterrupt: 

## 2. Entrenamiento Modelo MEDIUM (yolo12m)
**Batch Size:** 16 (Equilibrio memoria/rendimiento)

In [4]:
MODEL_ARCH = "yolo12m.pt"
BATCH_SIZE = 16
model_name_clean = "yolo12m"

print(f"\n\n{'#'*60}")
print(f" INICIANDO PROCESO PARA MODELO: {model_name_clean}")
print(f"{'#'*60}")

# --- FASE 1: PRE-ENTRENAMIENTO AÉREO ---
print(f"\n>>> [{model_name_clean}] FASE 1: ENTRENAMIENTO EN DOMINIO AÉREO (FUENTE)")

model_phase1 = YOLO(MODEL_ARCH)
model_phase1.train(
    data=str(yaml_aereo),
    epochs=EPOCHS_AEREO,
    imgsz=640,
    batch=BATCH_SIZE,
    project=PROJECT_NAME,
    name=f"phase1_aerial_{model_name_clean}",
    exist_ok=True,
    verbose=True, 
    optimizer='SGD',
    lr0=0.01
)

# Recuperar los pesos resultantes de la Fase 1
weights_phase1 = Path(PROJECT_NAME) / f"phase1_aerial_{model_name_clean}" / "weights" / "best.pt"
print(f"✅ [{model_name_clean}] Fase 1 Completada. Pesos 'Expertos en Aire' guardados en: {weights_phase1}")

# --- FASE 2: FINE-TUNING SUBMARINO ---
print(f"\n>>> [{model_name_clean}] FASE 2: FINE-TUNING EN DOMINIO SUBMARINO (OBJETIVO)")

if weights_phase1.exists():
    model_phase2 = YOLO(weights_phase1)

    # 3. Entrenar (Fine-Tuning)
    model_phase2.train(
        data=str(yaml_acuatico),
        epochs=EPOCHS_FINE_TUNE,
        imgsz=640,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=f"phase2_underwater_{model_name_clean}",
        exist_ok=True,
        optimizer='SGD',
        lr0=0.005,      # LR Bajo
        lrf=0.1,
        freeze=10,      # Congelar backbone
        augment=True    
    )

    # Guardar resultado final
    final_weights = Path(PROJECT_NAME) / f"phase2_underwater_{model_name_clean}" / "weights" / "best.pt"
    dest = OUTPUT_DIR / f"modelo-mixto-m.pt"
    
    if final_weights.exists():
        shutil.copy(final_weights, dest)
        print(f"\n✅ MODELO MIXTO SECUENCIAL GUARDADO: {dest}")

        # --- VALIDACIÓN FINAL ---
        print(f"\nValidando el nuevo modelo {model_name_clean}...")
        test_yaml = './dataset_yolo/dataset.yml' 
        try:
            metrics = model_phase2.val(split='test', data=test_yaml)
            print(f"[{model_name_clean}] mAP@50 Final: {metrics.box.map50:.4f}")
        except Exception as e:
            print(f"No se pudo validar automáticamente. Error: {e}")
    else:
        print("Error: No se generaron pesos en la Fase 2.")
else:
    print(f"❌ Error Crítico: No se encontró el modelo de la Fase 1 en {weights_phase1}")



############################################################
 INICIANDO PROCESO PARA MODELO: yolo12m
############################################################

>>> [yolo12m] FASE 1: ENTRENAMIENTO EN DOMINIO AÉREO (FUENTE)
New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.228 🚀 Python-3.9.13 torch-2.8.0+cu128 CPU (AMD Ryzen 7 5800X 8-Core Processor)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=temp_aereo.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, i


KeyboardInterrupt



## 3. Entrenamiento Modelo LARGE (yolo12l)
**Batch Size:** 8 (Modelo pesado, reducir batch para evitar OOM)

In [5]:
MODEL_ARCH = "yolo12l.pt"
BATCH_SIZE = 8
model_name_clean = "yolo12l"

print(f"\n\n{'#'*60}")
print(f" INICIANDO PROCESO PARA MODELO: {model_name_clean}")
print(f"{'#'*60}")

# --- FASE 1: PRE-ENTRENAMIENTO AÉREO ---
print(f"\n>>> [{model_name_clean}] FASE 1: ENTRENAMIENTO EN DOMINIO AÉREO (FUENTE)")

model_phase1 = YOLO(MODEL_ARCH)
model_phase1.train(
    data=str(yaml_aereo),
    epochs=EPOCHS_AEREO,
    imgsz=640,
    batch=BATCH_SIZE,
    project=PROJECT_NAME,
    name=f"phase1_aerial_{model_name_clean}",
    exist_ok=True,
    verbose=True, 
    optimizer='SGD',
    lr0=0.01
)

# Recuperar los pesos resultantes de la Fase 1
weights_phase1 = Path(PROJECT_NAME) / f"phase1_aerial_{model_name_clean}" / "weights" / "best.pt"
print(f"✅ [{model_name_clean}] Fase 1 Completada. Pesos 'Expertos en Aire' guardados en: {weights_phase1}")

# --- FASE 2: FINE-TUNING SUBMARINO ---
print(f"\n>>> [{model_name_clean}] FASE 2: FINE-TUNING EN DOMINIO SUBMARINO (OBJETIVO)")

if weights_phase1.exists():
    model_phase2 = YOLO(weights_phase1)

    # 3. Entrenar (Fine-Tuning)
    model_phase2.train(
        data=str(yaml_acuatico),
        epochs=EPOCHS_FINE_TUNE,
        imgsz=640,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=f"phase2_underwater_{model_name_clean}",
        exist_ok=True,
        optimizer='SGD',
        lr0=0.005,      # LR Bajo
        lrf=0.1,
        freeze=10,      # Congelar backbone
        augment=True    
    )

    # Guardar resultado final
    final_weights = Path(PROJECT_NAME) / f"phase2_underwater_{model_name_clean}" / "weights" / "best.pt"
    dest = OUTPUT_DIR / f"modelo-mixto-l.pt"
    
    if final_weights.exists():
        shutil.copy(final_weights, dest)
        print(f"\n✅ MODELO MIXTO SECUENCIAL GUARDADO: {dest}")

        # --- VALIDACIÓN FINAL ---
        print(f"\nValidando el nuevo modelo {model_name_clean}...")
        test_yaml = './dataset_yolo/dataset.yml' 
        try:
            metrics = model_phase2.val(split='test', data=test_yaml)
            print(f"[{model_name_clean}] mAP@50 Final: {metrics.box.map50:.4f}")
        except Exception as e:
            print(f"No se pudo validar automáticamente. Error: {e}")
    else:
        print("Error: No se generaron pesos en la Fase 2.")
else:
    print(f"❌ Error Crítico: No se encontró el modelo de la Fase 1 en {weights_phase1}")



############################################################
 INICIANDO PROCESO PARA MODELO: yolo12l
############################################################

>>> [yolo12l] FASE 1: ENTRENAMIENTO EN DOMINIO AÉREO (FUENTE)
New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.228 🚀 Python-3.9.13 torch-2.8.0+cu128 CPU (AMD Ryzen 7 5800X 8-Core Processor)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=temp_aereo.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, io

KeyboardInterrupt: 